# Regra de Simpson

In [1]:
from typing import Callable
import numpy as np
import matplotlib.pyplot as plt

# Regra de Simpson

In [2]:
def simpson(start: float, end: float, func: Callable, N: int, final: bool = False) -> (float | tuple[np.ndarray, np.ndarray]):
    """
        Calcula o integral de `func` de `start` a `end` com N passos usando a regra de Simpson. Devolve apenas o valor final se `final == True`.
    """
    
    # Verificar que N é par
    N = (N//2) * 2

    # Abcissas
    h = (end - start) / N
    tt = np.linspace(start, end, N + 1)
    vv = func(tt)

    # Área segmentos
    aa = vv[:-2:2] + 4 * vv[1:-1:2] + vv[2: :2]
    integral = h * (aa.cumsum()) / 3

    if final:
        return integral[-1]

    else:
        return tt, integral

# Regra de Simpson Adaptativa

In [3]:
def simpsonA(start: float, end: float, func: Callable, eps = 1e-6, maxN: int = 10_000) -> float:
    """
        Calcula o integral de `func` de `start` a `end` usando a regra de Simpson adaptativa começando com duas fatias e usando no máximo `maxN` fatias.
    """

    # Integral para N = 2
    N1 = 2
    h1 = (end - start) / N1

    S1 = (func(start) + func(end)) / 3
    T1 = 2 * func(start + h1) / 3
    I1 = h1 * (S1 + 2 * T1)

    # Usar o método adaptativo
    erro = eps + 1
    while abs(erro) > abs(eps):
        # Calcular o próximo integral
        N2 = N1 * 2
        h2 = h1 / 2

        S2 = S1 + T1
        T2 = 2 * np.sum(func(start + np.arange(1, N2 + 1, 2) * h2)) / 3     # O último elemento do arange é N2-1
        I2 = h2 * (S2 + 2 * T2)

        # Estimar o erro
        erro = (I2 - I1) / 15

        # print(f"Para N = {N2} o valor do integral é {I2} e o erro é cerca de {erro}")

        # O I2 atual é o próximo I1
        N1, h1, T1, S1, I1 = N2, h2, T2, S2, I2
        
        if N1 > maxN:
            raise ValueError(f"Não foi possível calcular o integral com a precisão desejada com menos de maxN = {maxN} fatias!")
    
    return I1

# Método de Romberg com Regra de Simpson

Neste programa estamos a usar o truque que desenvolvemos no método adaptativo para calcular integrais com o dobro dos passos para otimizar o cálculo de integrais!

In [4]:
def rombergS(start: float, end: float, func: Callable, eps: float = 1e-6, maxN: int = 10_000) -> float:
    """
        Calcula o integral de `func` de `start` a `end` usando integração de Romberg com base na regra do trapézio começando com duas fatias e usando no máximo `maxN` fatias.
    """

    # Matriz com as últimas duas linhas da matriz R_{i, j}
    RRnow = []
    RRprev = []

    # Calcular R_{0, 0}
    N1 = 2
    h1 = (end - start) / N1

    S1 = (func(start) + func(end)) / 3
    T1 = 2 * func(start + h1) / 3
    I1 = h1 * (S1 + 2 * T1)

    RRprev += [I1]

    
    # Queremos calcular R_{i, i}
    i = 1

    # Usar o método de Romberg
    erro = eps + 1
    while abs(erro) > abs(eps):
        # O número de passos que damos duplica
        N2 = N1 * 2
        h2 = h1 / 2

        # Calcular o novo integral
        S2 = S1 + T1
        T2 = 2 * np.sum(func(start + np.arange(1, N2 + 1, 2) * h2)) / 3
        I2 = h2 * (S2 + 2 * T2)

        # Guardar R_{i, 1}
        RRnow += [I2]


        # Calcular R_{i, j+1}, pelo que j+1 vai de 1 até i 
        for j in range(0, i):
            RRnow += [RRnow[j] + (RRnow[j] - RRprev[j]) / (4**(j+2) - 1)]
        
        # Estimar o erro de R_{i, i}
        erro = (RRnow[i-1] - RRprev[i-1]) / (4**(i+1)-1)

        # print(f"Para N = {N2} o valor do integral é {RR[i][i]} e o erro é cerca de {erro}")

        # O I2 atual é o próximo I1
        N1, h1, T1, S1, I1 = N2, h2, T2, S2, I2
        RRprev, RRnow = RRnow.copy(), []

        # Aumentar i
        i += 1

        if N1 > maxN:
            raise ValueError(f"Não foi possível calcular o integral com a precisão desejada com menos de maxN = {maxN} fatias!")
    
    return RRprev[i-1]

# Regra de Simpson 2D

In [5]:
def simpson2d(xlim: tuple[float, float], ylim: tuple[float, float], func: Callable, N: tuple[int, int]) -> (float | tuple[np.ndarray, np.ndarray]):
    """
        Calcula o integral de `func(x, y)` no bloco onde x está no intervalo `xlim` e y no intervalo `ylim` com `N[0]` passos segundo x e `N[1]` passos segundo y usando a regra de Simpson.
    """
    
    # Verificar que N é par
    Nx = (N[0]//2) * 2
    Ny = (N[1]//2) * 2

    # Abcissas
    hx = (xlim[1] - xlim[0]) / Nx
    hy = (ylim[1] - ylim[0]) / Ny

    # Grelha de pontos
    xx = np.linspace(xlim[0], xlim[1], Nx + 1)
    yy = np.linspace(ylim[0], ylim[1], Ny + 1)

    # Pesos de Simpson
    wx = np.ones_like(xx)
    wx[1:Nx:2] = 4
    wx[2:Nx:2] = 2
    wx *= hx / 3

    wy = np.ones_like(yy)
    wy[1:Ny:2] = 4
    wy[2:Ny:2] = 2
    wy *= hy / 3

    # Meshgrid da grelha e dos pesos
    xm, ym = np.meshgrid(xx, yy, sparse=True)
    wxm, wym = np.meshgrid(wx, wy, sparse=True)

    # Calcular o integral
    return np.sum(func(xm, ym) * wxm * wym)

# Testar

Vamos fazer uma bateria de testes a este métodos!

In [6]:
from testes import testeintegral

In [7]:
print("Regra de Simpson\n")
testeintegral(lambda start, end, func, N: simpson(start, end, func, N, True), 10)

Regra de Simpson

Integral de x de 0 a 1
Valor obtido: 0.5000000000000001.
Valor esperado: 0.5.
Diferença relativa: 2.22e-14%



Integral de x^2 de 0 a 1
Valor obtido: 0.3333333333333334.
Valor esperado: 0.3333333333333333.
Diferença relativa: 3.33e-14%



Integral de x^4 - 2x + 1 de 0 a 2
Valor obtido: 4.400426666666668.
Valor esperado: 4.4.
Diferença relativa: 9.70e-03%



Integral de sin(x) de 0 a pi
Valor obtido: 2.0001095173150043.
Valor esperado: 2.
Diferença relativa: 5.48e-03%


In [8]:
print("Regra de Simpson Adaptativa\n")
testeintegral(lambda start, end, func, N: simpsonA(start, end, func, 1e-6), 10)

# Em todos os casos o erro é de facto menor que eps!

Regra de Simpson Adaptativa

Integral de x de 0 a 1
Valor obtido: 0.5.
Valor esperado: 0.5.
Diferença relativa: 0.00e+00%



Integral de x^2 de 0 a 1
Valor obtido: 0.33333333333333337.
Valor esperado: 0.3333333333333333.
Diferença relativa: 1.67e-14%



Integral de x^4 - 2x + 1 de 0 a 2
Valor obtido: 4.4000002543131504.
Valor esperado: 4.4.
Diferença relativa: 5.78e-06%



Integral de sin(x) de 0 a pi
Valor obtido: 2.0000000645300022.
Valor esperado: 2.
Diferença relativa: 3.23e-06%


In [9]:
print("Romberg com Simpson\n")
testeintegral(lambda start, end, func, N: rombergS(start, end, func, eps=1e-2), 10)

# Em todos os casos o erro é de facto menor que eps!

Romberg com Simpson

Integral de x de 0 a 1
Valor obtido: 0.5.
Valor esperado: 0.5.
Diferença relativa: 0.00e+00%



Integral de x^2 de 0 a 1
Valor obtido: 0.33333333333333337.
Valor esperado: 0.3333333333333333.
Diferença relativa: 1.67e-14%



Integral de x^4 - 2x + 1 de 0 a 2
Valor obtido: 4.3999999999999995.
Valor esperado: 4.4.
Diferença relativa: 2.02e-14%



Integral de sin(x) de 0 a pi
Valor obtido: 1.9985707318238362.
Valor esperado: 2.
Diferença relativa: 7.15e-02%


In [10]:
from testes import testeintegral2d

print("Regra de Simpson 2D\n")
testeintegral2d(simpson2d, (100, 100))

Regra de Simpson 2D

Integral de x num quadrado com x, y em [0, 1]
Valor obtido: 0.5000000000000001.
Valor esperado: 0.5.
Diferença relativa: 2.22e-14%



Integral de x^2 + y^2 num quadrado com x, y em [0, 1]
Valor obtido: 0.6666666666666667.
Valor esperado: 0.6666666666666666.
Diferença relativa: 1.67e-14%



Integral de xy + sin(x) + cos(2y) num quadrado com x, y em [0, 1]
Valor obtido: 1.1643464079743917.
Valor esperado: 1.1643464075447012.
Diferença relativa: 3.69e-08%



